In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, mid_channels, stride=1):
        """
        in_channels:輸入通道數
        mid_channels:中間及輸出通道數
        """
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels = in_channels,
            out_channels = mid_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False,
        )
        self.bn1 = nn.BatchNorm2d(mid_channels)
        self.conv2 = nn.Conv2d(
            mid_channels,
            mid_channels,
            kernel_size=3,
            stride=1,
            padding=1,
            bias=False
        )
        self.bn2 = nn.BatchNorm2d(mid_channels)
        #定義短接網路，如果沒有調整維度，shortcut就是一個空的nn.Sequential
        self.shortcut = nn.Sequential()
        #因為shortcut後需要將兩個分支累加，所以要求兩個分支的維度匹配
        #所以input_channels與最終的channels布匹配時，需要透過1*1卷積進行
        if stride != 1 or in_channels != mid_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(
                    in_channels,
                    mid_channels,
                    kernel_size=1,
                    stride=stride,
                    bias=False,
                ),
                nn.BatchNorm2d(mid_channels),
            )
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [3]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        # 對啟動值進行正歸化，使網路梯度在反向傳播時不會受到參數尺度影響
        self.bn1 = nn.BatchNorm2d(64)
        # 架設 basicblock
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        # 最後的線性層
        self.linear = nn.Linear(512, num_classes)
        
    def _make_layer(self, block, mid_channels, num_blocks, stride):
        #stride 僅指定第一個block的stride，後面都預設為1
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, mid_channels, stride))
            self.in_channels = mid_channels
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [4]:
def resnet18():
    #[2,2,2,2]代表Block的重複堆疊次數
    return ResNet(BasicBlock, [2,2,2,2])
    

In [6]:
if __name__ == '__main__':
    from torchsummary import summary
    
    resnet = resnet18().cuda()
#     summary(resnet,(3, 32, 32))